In [ ]:
%pip install vulncheck-sdk matplotlib pandas

In [ ]:
import vulncheck_sdk
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

DEFAULT_HOST = "https://api.vulncheck.com"
DEFAULT_API = DEFAULT_HOST + "/v3"
TOKEN = "INSERT YOUR TOKEN"  # Replace with your actual API token

# Configure the VulnCheck API client
configuration = vulncheck_sdk.Configuration(host=DEFAULT_API)
configuration.api_key["Bearer"] = TOKEN

with vulncheck_sdk.ApiClient(configuration) as api_client:
    indices_client = vulncheck_sdk.IndicesApi(api_client)

    # Initialize the body of the request with start_cursor=True to start pagination
    body = vulncheck_sdk.ParamsIdxReqParams()

    # Initialize lists to store vendor and CVE data
    cve = []
    vendor = []
    product = []
    ransomware = []
    date_added = []

    # Make the initial request to start pagination
    api_response = indices_client.index_vulncheck_kev_get(body,start_cursor="true")

   # Process the first page of results
for entry in api_response.data:
    # Directly access the first element of the list
    cve.append(entry.cve[0])  # Since there is always one CVE in the list
    
    vendor.append(entry.vendor_project)
    product.append(entry.product)
    ransomware.append(entry.known_ransomware_campaign_use)
    date_added.append(entry.date_added)

# Continue fetching data while there's a next cursor
while api_response.meta.next_cursor is not None:
    print(f"Fetching next page with cursor: {api_response.meta.next_cursor}")

    # Fetch the next page using the cursor
    api_response = indices_client.index_vulncheck_kev_get(
        vulncheck_sdk.ParamsIdxReqParams(), cursor=api_response.meta.next_cursor
    )

    # Append the new data from the next page
    for entry in api_response.data:
        cve.append(entry.cve[0])  # Since there is always one CVE in the list
        
        vendor.append(entry.vendor_project)
        product.append(entry.product)
        ransomware.append(entry.known_ransomware_campaign_use)
        date_added.append(entry.date_added)
        


# Create a DataFrame from the accumulated data
df = pd.DataFrame({
    'CVE': cve,
    'Vendor': vendor,
    'Product': product,
    'Ransomware': ransomware,
    'Date Added': date_added

})

# Display the DataFrame
df



In [ ]:

# Count the unique CVEs per vendor
vendor_cve_count = df.groupby('Vendor')['CVE'].nunique().reset_index()

# Rename columns for clarity
vendor_cve_count.columns = ['Vendor', 'CVE']

# Sorting by the number of unique CVEs in descending order for better visualization
vendor_cve_count = vendor_cve_count.sort_values(by='CVE', ascending=False)

# Limiting the data to the top 50 vendors by the number of unique CVEs
top_50_vendors = vendor_cve_count.head(50)

# Plotting the horizontal bar chart with the highest CVE count at the top
plt.figure(figsize=(12, 10))  # Adjusting size for better readability
plt.barh(top_50_vendors['Vendor'], top_50_vendors['CVE'], color='blue')  # Horizontal bar chart

# Reverse the order of y-axis (so the highest value is at the top)
plt.gca().invert_yaxis()

# Set the title and axis labels
plt.title('Top 50 Vendors by Number of KEVs')
plt.xlabel('Number of KEVs')  # CVE count on x-axis
plt.ylabel('Vendor')  # Vendors on y-axis
plt.xticks(fontsize=10)  # Adjust font size for x-ticks
plt.yticks(fontsize=8)  # Adjust font size for y-ticks
plt.tight_layout()

# Show the plot
plt.show()